# Case Study Answers

In [2]:
# Import packages 
import os 
import glob
import numpy as np 
import pandas as pd 
import plotly.express as px 
import plotly.graph_objects as go 

from pathlib import Path

In [3]:
file_path = r"/BMCC/Lesson Materials/Weeks 6 - 7/Data/Instacart Market Basket Analysis"
os.chdir(file_path)
sourcefiles = os.listdir(file_path)

In [4]:
# 1 - Import all datasets as dataframes using dictionary comprehension
# Functionality of .resolve():
# Converts relative paths to absolute paths: Given a relative path, .resolve() resolves it to its corresponding absolute path. This means it determines the full path to the file or directory by considering the current working directory.
# Normalizes paths: It also normalizes the path, removing any redundant components like "." (current directory) and ".." (parent directory).
dfs = {file.stem: pd.read_csv(file.resolve()) for file in Path(file_path).glob("*.csv")}

In [5]:
products_df = dfs["products"]
orders_df = dfs["orders"]
departments_df = dfs["departments"]
aisles_df = dfs["aisles"]
order_products_prior_df = dfs["order_products__prior"]

In [ ]:
# 2 - Print the shape of each dataframe 
print(f"Shape of products dataframe: {products_df.shape}\n")
print(f"Shape of orders dataframe: {orders_df.shape}\n")
print(f"Shape of departments dataframe: {departments_df.shape}\n")
print(f"Shape of aisles dataframe: {aisles_df.shape}\n")
print(f"Shape of order products prior dataframe: {order_products_prior_df.shape}\n")

In [30]:
# 3 - Merge dataframes to show products, aisles, and departments they are found in 
merge1 = pd.merge(products_df, aisles_df, on = "aisle_id", how = "inner")
final_merge1 = pd.merge(merge1, departments_df, on = "department_id", how = "inner")

In [31]:
# 4 - Merge the orders, products, and order_products_prior dataframes on "order_id" and "product_id"
# 4a - Perform this merge using the inner and left methods 
# 4b - What is the shape of both dataframes af the merge 

# INNER MERGE 
# Merge orders and order_products_prior first 
merge1 = pd.merge(orders_df, order_products_prior_df, on ="order_id", how = "inner")
final_merge2 = pd.merge(merge1, products_df, on = "product_id", how = "inner")

# 4b - What is the shape of both dataframes af the merge
# final_merge2.shape

In [ ]:
# LEFT MERGE
merge2 = pd.merge(orders_df, order_products_prior_df, on = "order_id", how = "left")
final_merge3 = pd.merge(merge2, products_df, on = "product_id", how = "left")

# 4b - What is the shape of both dataframes af the merge
final_merge3.shape

In [ ]:
# 5 - Perform EDA on the newly merged dataframes that were a result of the inner joins 
# 5a - Print the summary stats for the merged dataframes 
print("SUMMARY STATS FOR ORDER, AISLES, AND PRODUCTS DATAFRAME")
print(final_merge1.describe())

In [ ]:
print("SUMMARY STATS FOR ORDER PRODUCTS PRIOR DATAFRAME")
print(final_merge2.describe())

In [37]:
# 6 - VISUALIZATIONS 
# 6a - Bar plot to visualize most ordered product in the data 

# Prep data 
most_ordered_prod = final_merge2.groupby("product_name").agg(Count_orders = pd.NamedAgg("product_name", "size")).sort_values("Count_orders", ascending = False).reset_index(drop = False)
most_ordered_prod_10 = most_ordered_prod.head(10)

In [ ]:
# Create bar plot 
# colors = ["#88AB75", "#2D93AD", "#7D7C84"]
bar_plot = go.Bar(
    x = most_ordered_prod_10["product_name"], 
    y = most_ordered_prod_10["Count_orders"],
    marker = dict(color = "#2D93AD"),
    text = most_ordered_prod_10["Count_orders"],
    textposition = "outside"
)

# Create a layout for the bar plot
layout = go.Layout(
    title = "Top 10 Most Ordered Products",
    xaxis_title = "Product Name", 
    yaxis_title = "Number of Orders",
    height = 600,
    width = 800
)

# View the barplot using the layout from above
bar_fig = go.Figure(data = [bar_plot], layout = layout)
bar_fig.show()

In [ ]:
# 6b - Bar plot to visualize the most reordered product in the data 
# Create dataframe to show the most reordered product 

reorders = final_merge2[final_merge2["reordered"] == 1]
reorders.head()

In [45]:
# Reorders groupby 
reorders_grouped = reorders.groupby("product_name").agg(Order_count = pd.NamedAgg("product_name", "size")).sort_values("Order_count", ascending =  False).reset_index(drop = False)
reorders_grouped_10 = reorders_grouped.head(10)

In [ ]:
# Create bar plot 
# colors = ["#88AB75", "#2D93AD", "#7D7C84"]
bar_plot = go.Bar(
    x = reorders_grouped_10["product_name"], 
    y = reorders_grouped_10["Order_count"],
    marker = dict(color = "#2D93AD"),
    text = reorders_grouped_10["Order_count"],
    textposition = "outside"
)

# Create a layout for the bar plot
layout = go.Layout(
    title = "Top 10 Most Re-Ordered Products",
    xaxis_title = "Product Name", 
    yaxis_title = "Number of Orders",
    height = 600,
    width = 800
)

# View the barplot using the layout from above
bar_fig = go.Figure(data = [bar_plot], layout = layout)
bar_fig.show()

In [ ]:
# 6c - Most Ordered Departments or Aisles: Show the total number of orders by department (e.g., produce, dairy) or 
# aisles to understand which departments are the most popular

# Look at columns names to figure out what to merge
final_merge2.columns

In [ ]:
# Merge datasets
department_merge_final = pd.merge(final_merge2, departments_df, on = "department_id", how = "inner")
aisle_merge_department = pd.merge(department_merge_final, aisles_df, on = "aisle_id", how = "inner")

aisle_merge_department.head(10)

In [ ]:
# Group aisles 
aisles_grouped_10 = aisle_merge_department.groupby("aisle").agg(Count_aisle = pd.NamedAgg("aisle", "size")).sort_values("Count_aisle", ascending = False).reset_index(drop = False).head(10)
aisles_grouped_10

In [ ]:
# Create bar plot 
# colors = ["#88AB75", "#2D93AD", "#7D7C84"]
bar_plot = go.Bar(
    x = aisles_grouped_10["aisle"], 
    y = aisles_grouped_10["Count_aisle"],
    marker = dict(color = "#2D93AD"),
    text = aisles_grouped_10["Count_aisle"],
    textposition = "outside"
)

# Create a layout for the bar plot
layout = go.Layout(
    title = "Most Commonly Shopped Aisle",
    xaxis_title = "Aisle Name", 
    yaxis_title = "Number of Orders",
    height = 600,
    width = 800
)

# View the barplot using the layout from above
bar_fig = go.Figure(data = [bar_plot], layout = layout)
bar_fig.show()

In [55]:
# 6d - Encode the day of week column (dow) from numerical to categorical (the day name) and produce a visualization 
# that shows all orders by day of week and sort the visualization in ascending order based on day of week 

# Use the calendar module 
import calendar

In [56]:
# Create function to 'translate' dow to day name 
def day_name(weekday_int):
    return calendar.day_name[weekday_int]

In [57]:
# Create day name column 
aisle_merge_department["Day Name"] = aisle_merge_department["order_dow"].apply(lambda x: day_name(x))

In [ ]:
# Create dataframe that counts orders by day of week 
days_grouped = aisle_merge_department.groupby(["order_dow", "Day Name"]).agg(Orders = pd.NamedAgg("Day Name", "size")).sort_values("order_dow", ascending = True).reset_index(drop = False)

days_grouped.dtypes

In [ ]:
days_grouped["Weekday concat"] = days_grouped["order_dow"].astype(str) + "-" + days_grouped["Day Name"]
days_grouped

In [ ]:
# Create bar plot 
# colors = ["#88AB75", "#2D93AD", "#7D7C84"]
bar_plot = go.Bar(
    x = days_grouped["Weekday concat"], 
    y = days_grouped["Orders"],
    marker = dict(color = "#2D93AD"),
    text = days_grouped["Orders"],
    textposition = "outside"
)

# Create a layout for the bar plot
layout = go.Layout(
    title = "Order Counts by Day of Week",
    xaxis_title = "Day Name", 
    yaxis_title = "Number of Orders",
    height = 600,
    width = 800
)

# View the barplot using the layout from above
bar_fig = go.Figure(data = [bar_plot], layout = layout)
bar_fig.show()

In [ ]:
aisle_merge_department.columns

In [ ]:
# 6e - What hour of the day are customers most likely to order a product? 
order_hour = aisle_merge_department.groupby("order_hour_of_day").agg(Order = pd.NamedAgg("order_hour_of_day", "size")).sort_values("order_hour_of_day", ascending = True).reset_index(drop = False)
order_hour

In [78]:
#  6f - A word cloud of products to show the most ordered product
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
most_ordered = most_ordered_prod_10.set_index("product_name").to_dict()["Count_orders"]
most_ordered

In [ ]:
wc = WordCloud(width = 800, height = 400).generate_from_frequencies(most_ordered)

plt.figure(figsize = (10, 10))
plt.imshow(wc, interpolation = 'bilinear')
plt.axis('off')
plt.show()


In [ ]:
aisle_merge_department.columns

In [ ]:
# 6g - Hourly/Weekly Order Patterns: Create a heatmap showing the frequency of orders at different times of the day and 
# days of the week. This can reveal peak shopping hours and days.

# Create dataframe of hours per hour per day 
day_hour = aisle_merge_department.groupby(["order_hour_of_day", "Day Name"]).agg(Count = pd.NamedAgg("order_hour_of_day", "size")).sort_values("order_hour_of_day", ascending = True).reset_index(drop = False)
day_hour

In [ ]:
# Pivot data to create matrix 
heatmap_data = day_hour.pivot(index = "order_hour_of_day", columns = "Day Name", values = "Count")
heatmap_data

In [ ]:
# Create heatmap 
fig = go.Figure(data = go.Heatmap(
    z = heatmap_data.values,
    x = heatmap_data.columns, 
    y = heatmap_data.index,
    type = "heatmap",
    colorscale = "rdbu"
))

fig.update_layout(title = "Heatmap of Orders by Day of Week and Hour")

fig.update_layout(xaxis = dict(title = "Day of Week", zeroline = True, zerolinecolor = 'red', zerolinewidth = 2))

# Add zeroline to y-axis
fig.update_layout(yaxis = dict(title = "Hour of Day", zeroline = True, zerolinecolor = 'red', zerolinewidth = 2))

fig.show()

In [ ]:
# Reorder Probability Heatmap: Visualize the likelihood of 
# products being reordered by plotting the reorder probability 
# for different products or departments

# DATAFRAMES: 
    # products_df = dfs["products"]
    # orders_df = dfs["orders"]
    # departments_df = dfs["departments"]
    # aisles_df = dfs["aisles"]
    # order_products_prior_df = dfs["order_products__prior"]

products_df.columns

In [ ]:


# Group by product_id to calculate total orders and total reorders
product_reorders = order_products_prior_df.groupby('product_id')['reordered'].agg(['sum', 'count']).reset_index()

# Rename columns for clarity
product_reorders.columns = ['product_id', 'total_reorders', 'total_orders']

# Calculate reorder probability
product_reorders['reorder_probability'] = product_reorders['total_reorders'] / product_reorders['total_orders']

# Display the result
product_reorders[['product_id', 'reorder_probability']].head()

# Merge product information with reorder probabilities
product_reorders = pd.merge(product_reorders, products_df[['product_id', 'department_id']], on = 'product_id')

# Group by department_id and product_id to calculate reorder probability per product in each department
department_reorder = product_reorders.groupby(['department_id', 'product_id'])['reorder_probability'].mean().reset_index()
department_reorder = department_reorder.sort_values("reorder_probability", ascending = False).reset_index(drop = True)
department_reorder_100 = department_reorder.head(100)

# Pivot data to create a matrix (department_id as rows, product_id as columns)
heatmap_data = department_reorder_100.pivot(index = 'department_id', columns = 'product_id', values = 'reorder_probability')

# Create a heatmap using Plotly's go.Heatmap
fig = go.Figure(data = go.Heatmap(
    z = heatmap_data.values,      # Reorder probabilities
    x = heatmap_data.columns,     # Product IDs
    y = heatmap_data.index,       # Department IDs
    colorscale = 'rdbu',       # Change colorscale if desired
    colorbar = dict(title = 'Reorder Probability')  # Title for the color bar
))

# Add title and labels
fig.update_layout(
    title = 'Reorder Probability Heatmap by Department and Product',  # Number of ticks on the x-axis
    xaxis_title = "Product ID",
    yaxis_title = "Department ID",
    autosize = False,
    width = 900,
    height = 700
)

# Display the heatmap
fig.show()


In [130]:
# Proportion of Reordered vs. New Products: Stack orders by whether 
# they are reorders or first-time purchases to observe reorder trends 
# across departments

# Merge ordered products with products and department data 
order_products_prior_df = pd.merge(order_products_prior_df, 
                                   products_df[["product_id", "department_id"]], 
                                   on = "product_id", how = "inner")
order_products_prior_df = pd.merge(order_products_prior_df, 
                                   departments_df[["department_id", "department"]],
                                   on = "department_id", how = "inner")

In [131]:
# Groupby department and reordered to get counts of each departments' new and reordered statuses
department_orders = order_products_prior_df.groupby(["department", "reordered"]).size().reset_index(name = "counts")

In [ ]:
# Calculate proportions and map 0 and 1 in reordered column to "Reorder" and "New Order"
department_orders["proportion"] = department_orders.groupby("department")["counts"].transform(lambda x: x / x.sum())
department_orders["reordered"] = department_orders["reordered"].map({1: "Reorder", 0: "New Order"})

department_orders.head(10)

In [ ]:
# Create stacked bar chart to display proportions by department
fig = px.bar(department_orders, 
             x = "department", 
             y = "proportion", 
             color = "reordered",
             title = "Proportion of Reordered vs. Newly Ordered Products by Department",
             labels = {"proportion": "Proportion", "department": "Department", "reordered": "Reordered"},
             color_discrete_sequence = ["#88AB75", "#2D93AD"])

fig.show()

In [ ]:
orders_df.columns

In [ ]:
# Customer Behavior by Order Size: A histogram or density plot showing the distribution of order sizes (number of items per order). 
# Segment by customers to identify groups with smaller or larger basket sizes

# Create merged dataframe to get order information for customers 
order_data = pd.merge(order_products_prior_df, orders_df[["order_id", "user_id", "order_number"]], 
                      on = "order_id")

# Create a new 'customer_type' column 
# 'New' = first ordering customer 
# 'Repeat' = multiple orders
order_data["customer_type"] = order_data["order_number"].apply(lambda x: "New" if x == 1 else "Repeat")


In [ ]:
# Calculate order size (number of products) per order 
order_size = order_data.groupby(["order_id", "customer_type"]).size().reset_index(name = "order_size")

# Create histogram to compare order size distribution for new vs repeat customers 
fig = px.histogram(order_size,
                   x = "order_size",
                   color = "customer_type",
                   nbins = 30,
                   barmode = "overlay", 
                   title = "Customer Behavior: Order Size Distribution (New vs. Repeat Customer)",
                   labels = {"order_size": "Order Size (Number of Products)", "count": "Count", "customer_type": "Customer Type"},
                   color_discrete_sequence = ["#88AB75", "#2D93AD"])

fig.show()

In [ ]:
# Visualize via box plot 
fig = px.box(order_size, 
             x = "customer_type", 
             y = "order_size", 
             title = "Customer Behavior: Order Size Distribution (New vs. Repeat Customers",
             labels = {"order_size": "Order Size (Number of Products)", "customer_type": "Customer Type"},
             color = "customer_type", 
             color_discrete_sequence = ["#88AB75", "#2D93AD"])

fig.show()

In [6]:
# Loyalty Analysis: Bar plots showing the percentage of reorders by 
# customers to understand customer loyalty

# Merge order_products_prior with orders to get customer information
order_data = pd.merge(order_products_prior_df, orders_df[["order_id", "user_id"]],
                      on = "order_id", how = "inner")

# Calculate reorder percentage for each customer
customer_reorders = order_data.groupby("user_id").agg(
    total_orders = ("reordered", "size"),
    reorder_count = ("reordered", "sum")
).reset_index()

In [7]:
# Calculate percentage of reorders (reorder_count / total_orders)
customer_reorders["reorder_percentage"] = (customer_reorders["reorder_count"] / customer_reorders["total_orders"]) * 100


In [8]:
# Create bins for reorder percentages (0-10%, 10-20%, etc)
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-10%', '10-20%', '20-30%', '30-40%', '40-50%', '50-60%', '60-70%', '70-80%', '80-90%', '90-100%']
customer_reorders["reorder_bin"] = pd.cut(customer_reorders["reorder_percentage"], bins = bins, labels = labels, include_lowest = True)


In [ ]:
# Groupby reorder bins to get count of customers in each bin 
reorder_dist = customer_reorders["reorder_bin"].value_counts().sort_index().reset_index()
reorder_dist.columns = ["Reorder Percentage", "Customer Count"]
reorder_dist

In [ ]:
# Create bar plot to visualize percentage of reorders by customers 
fig = px.bar(reorder_dist, 
             x = "Reorder Percentage",
             y = "Customer Count",
             title = "Customer Loyalty: Percentage of Reorders by Customers", 
             labels = {"Customer Count": "Number of Customers", "Reorder Percentage": "Reorder Percentage Range"},
             color_discrete_sequence = ["#88AB75"])

fig.show()